Regis MSDS Deep Learning Project

This project is to create a deep learning model using the 100-bird-species data set. In this experiment I try two different methods for analyzing the picture data in an attempt to achieve 80% or greater accuracy.

The Data:

The data for this assignment is a living set of image data that is being updated regularly. There are many different images of a wide variety of birds but some of the species only have a few pictures. At the time of completing this project there are 190 species in the dataset. 

Summary of Methods:

To account for some of the bird species not having many pictures I used ImageDataGenerator to create more data for the model. Then I used the ImageDataGenerator to create train, validation, and test data. When compiling the model I used adam optimizer and learning rate optimization. I plotted the accuracy/loss vs epochs.

In the second model I used transfer learning from the MobileNetV2 model to import weights. I left out the Adam learning rate optimizer but still plotted the accuracy/loss vs epochs. 

Summary of Models:

In the first experiment I created a model using 2d convnets, max pooling and batch normalization layers. I used padding to ensure that the image data did not get to small as it was fed through the network. The data then is flattened then run through a Dense layer, a BatchNormalization, a dropout layer and then the output layer.

Next, I created a transfer learning model using the MobileNetV2. I kept the last four layers of the MobileNetV2 trainable and froze the rest and added some convolutional layers along with BatchNormalization layers.

Finally, I tried a functional API model but it took so long to process the data that kaggle ended up timing out

Analysis of results:

The model I created did pretty well with an accuracy of 85.9% but the MobileNetV2 only had an accuracy of 75.2%. I can only guess that this was because I did not include padding in this transfer learning model and the image data was reduced to a size to small for interpretation.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
    

# Any results you write to the current directory are saved as output.

First we will import the necessary Librarys to complete the project

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import backend, models, layers, optimizers, regularizers
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, SeparableConv2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from IPython.display import display  
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import os, shutil 
import random

import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.applications import MobileNetV2

In [ ]:
#We will set the seed to 42 for reproducibility
# Next we bring in the base directory for the data and define the paths
# we define a training Validation and a test path
# since this is a living data set we are working with
## the birdtypes is used so that if the set is updated the program should still work
np.random.seed(42)
base_dir = '/kaggle/input/100-bird-species'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')
birdtypes = os.listdir(train_dir)

In [ ]:
# Next we use ImageDataGenerator to create more data from the images we have
# we create a separate Data Generator for the test
## data because we do not want to augment this
train_datagen = ImageDataGenerator(rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(224, 224),
    batch_size=64,  
    class_mode= 'categorical')  

validation_data = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
def birds(DIR, dataset):
    label = []
    image = []
    j=0
    for i in range (0,30):
        j = random.randint(0, len(dataset.filenames))
        label.append(dataset.filenames[j].split('/')[0])
        image.append(DIR + '/' + dataset.filenames[j])
    return [label,image]

y,x = birds(train_dir, train_data)

for i in range(0, 16):
    X = load_img(x[i])
    plt.subplot(4,4,+1+i)
    plt.axis(False)
    plt.title(y[i], fontsize=8)
    plt.imshow(X)
plt.show()

In [ ]:
print('train_epochs', len(train_data)) # might be good to determine steps_per_epoch
print('valid_steps', len(validation_data)) # help determine Validation steps
print('test_data', len(test_data))

In [ ]:
backend.clear_session()

model = models.Sequential()
model.add(layers.Conv2D(32, (3 ,3), padding='same', activation ='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(layers.Conv2D(32, (3,3), activation ='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(layers.Conv2D(32, (3,3), padding='same', activation ='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(layers.Conv2D(64, (3 ,3), activation ='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(layers.Conv2D(64, (3,3), padding='same', activation ='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(layers.Conv2D(64, (3,3), padding='same', activation ='relu'))
model.add(BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(256, activation ='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(birdtypes), activation='softmax'))

model.summary()

In [ ]:
model.compile (optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

history = model.fit_generator(train_data,
                    epochs = 40,
                    steps_per_epoch = 404,
                    validation_steps= 30,
                    validation_data = (validation_data),
                    verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

test_loss, test_acc = model.evaluate_generator(test_data, steps=30)

print('model_test_acc:', test_acc)
print('Doesnt seem to work as well')

Here we can see this model has achieved an accuracy of 85.9%. Next, I imported the MobileNetV2 to try a transfer learning experiment.

In [ ]:
backend.clear_session()
mobmodel = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3))

for layer in mobmodel.layers[:-4]:
    layer.trainable = False
for layer in mobmodel.layers:
    print(layer, layer.trainable)


In [ ]:

model2 = models.Sequential()
model2.add(mobmodel)
model2.add(layers.Conv2D(64, kernel_size=3, activation ='relu'))
model2.add(BatchNormalization())

model2.add(layers.Conv2D(64, kernel_size=3, activation ='relu'))
model2.add(BatchNormalization())

model2.add(layers.Conv2D(64, kernel_size=3, activation ='relu'))
model2.add(BatchNormalization())

model2.add(layers.Flatten())
model2.add(layers.Dense(512, activation ='relu'))
model2.add(layers.Dense(len(birdtypes), activation='softmax'))

model2.summary()

In [ ]:
model2.compile (optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

history = model2.fit_generator(train_data,
                    epochs = 40,
                    steps_per_epoch = 404,
                    validation_steps= 30,
                    validation_data = (validation_data),
                    verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

test_loss, test_acc = model2.evaluate_generator(test_data, steps=30)

print('model_test_acc:', test_acc)
print('Doesnt seem to work as well')

In [ ]:
backend.clear_session()

#Visible layer
vislayer = Input(shape=(224,224,3))

#conv1
conv11 = SeparableConv2D(32, 3, padding='same', activation='relu')(vislayer)
conv12 = SeparableConv2D(64, 3,strides=2, padding='same', activation='relu')(conv11)
conv13 = MaxPooling2D((2,2), padding='same')(conv12)

#Conv2
conv21 = SeparableConv2D(32, 3, padding='same', activation='relu')(vislayer)
conv22 = SeparableConv2D(64, 3, padding='same', activation='relu', strides=2)(conv21)
conv23 = MaxPooling2D((2,2), padding='same')(conv22)

#Conv3
conv31 = SeparableConv2D(32, 3, activation='relu', padding='same')(vislayer)
conv32 = SeparableConv2D(64, 3,activation='relu', padding='same', strides=2)(conv31)
conv33 = MaxPooling2D((2,2), padding='same')(conv32)

#Conv4
conv42 = SeparableConv2D(32, 3, padding='same', activation='relu')(vislayer)
conv43 = SeparableConv2D(64, 3, padding='same', activation='relu', strides=2)(conv42)
conv44 = MaxPooling2D((2,2), padding='same')(conv43)

merged = Concatenate(axis=-1)([conv13, conv23, conv33, conv44])
lvl2norm = BatchNormalization()(merged)

#conv5
conv51 = SeparableConv2D(32, 3, padding='same', activation='relu')(lvl2norm)
conv52 = SeparableConv2D(64, 3,strides=2, padding='same', activation='relu')(conv51)
conv53 = MaxPooling2D((2,2), padding='same')(conv52)

#Conv6
conv61 = SeparableConv2D(32, 3, padding='same', activation='relu')(lvl2norm)
conv62 = SeparableConv2D(64, 3, padding='same', activation='relu', strides=2)(conv61)
conv63 = MaxPooling2D((2,2), padding='same')(conv62)

#Conv7
conv71 = SeparableConv2D(32, 3, activation='relu', padding='same')(lvl2norm)
conv72 = SeparableConv2D(64, 3,activation='relu', padding='same', strides=2)(conv71)
conv73 = MaxPooling2D((2,2), padding='same')(conv72)

#Conv8
conv82 = SeparableConv2D(32, 3, padding='same', activation='relu')(lvl2norm)
conv83 = SeparableConv2D(64, 3, padding='same', activation='relu', strides=2)(conv82)
conv84 = MaxPooling2D((2,2), padding='same')(conv83)

#concat
merged2 = Concatenate(axis=-1)([conv53, conv63, conv73, conv84])


#bnorm = BatchNormalization()(merged)
conv5 = Conv2D(32, 1, padding='same', activation='relu')(merged2)
conv6 = Conv2D(64, 3, padding='same', activation='relu')(conv5)
bnorm2 = BatchNormalization()(conv6)
flat = Flatten()(bnorm2)
hidden = Dense(256, activation='relu')(flat)
drop = Dropout(0.5)(hidden)
output = Dense(len(birdtypes), activation='softmax')(drop)

#Model
model3 = Model(inputs=vislayer, outputs=output)

plot_model(model3)

In [ ]:
#Functional API summary
model3.summary()

In [ ]:
model3.compile (optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

history = model3.fit_generator(train_data,
                    epochs = 40,
                    steps_per_epoch = 404,
                    validation_steps= 30,
                    validation_data = (validation_data),
                    verbose = 1,
                    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

test_loss, test_acc = model3.evaluate_generator(test_data, steps=30)

print('model_test_acc:', test_acc)
print('Doesnt seem to work as well')